In [40]:
import pandas as pd
import numpy as np
from sklearn import cross_validation

## Calculate median value based on training set

In [62]:
train = pd.read_csv("../data/train.csv", dtype={'StateHoliday': str}, parse_dates=["Date"])
train = train.loc[train.Sales > 0]
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
columns = ['Store', 'DayOfWeek', 'Promo', 'day']

## Prepare solution data with median value

In [69]:
medians = train.groupby( columns )['Sales'].median()
medians = medians.reset_index()
test = pd.read_csv("../data/test.csv", dtype={'StateHoliday': str}, parse_dates=["Date"])
test.loc[ test.Open.isnull(), 'Open' ] = 1
test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test2 = pd.merge( test, medians, on = columns, how = 'left' )
assert( len( test2 ) == len( test ))
test2.loc[ test2.Open == 0, 'Sales' ] = 0

test2[['Id', 'Sales']].astype(int).to_csv('../data/solution.csv', index=False)

ValueError: Cannot convert NA to integer

## Cross validate

In [63]:
(cv_train, cv_test) = cross_validation.train_test_split(train)
cv_medians = cv_train.groupby( columns )['Sales'].median()
cv_medians = cv_medians.reset_index()
cv_medians.rename(columns={'Sales': 'Sales2'}, inplace=True)
cv_test = pd.merge( cv_test, cv_medians, on = columns, how = 'left' )

spe = ((cv_test['Sales']-cv_test['Sales2'])/cv_test['Sales'])**2.0
mspe = spe.sum()/len(spe)
rmspe = np.sqrt(mspe)
print("RMSPE=%f" % rmspe)

RMSPE=0.192526
